# 1. Hypertuning 1D CNN
Study the pytorch documentation for:
- Dropout https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
- normalization layers https://pytorch.org/docs/stable/nn.html#normalization-layers

Experiment with adding dropout and normalization layers to your model. Some rough guidelines where to add them relative to Linear or Conv2d layers:
- Dropout: after Linear or Conv2d layers. Often added after the last Linear layer *before* the output layer, but could occur more often.
- Normalization layers: right after (blocks of) Linear or Conv2d layers, but before activation functions.

In [1]:
from pathlib import Path
import torch
import torch.nn as nn
from loguru import logger
import warnings
warnings.simplefilter("ignore", UserWarning)

In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor

for dataset in DatasetType:
    print(dataset)

DatasetType.FLOWERS
DatasetType.IMDB
DatasetType.GESTURES
DatasetType.FASHION
DatasetType.SUNSPOTS
DatasetType.IRIS
DatasetType.PENGUINS
DatasetType.FAVORITA
DatasetType.SECURE


In [3]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

flowersfactory = DatasetFactoryProvider.create_factory(DatasetType.FLOWERS)
streamers = flowersfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2024-12-13 16:33:08.336 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\Francesca\.cache\mads_datasets\flowers


In [4]:
len(train), len(valid)

(91, 22)

In [5]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))
x.shape, y.shape

(torch.Size([32, 3, 224, 224]), torch.Size([32]))

In [6]:
import torch
from torch import nn
from loguru import logger
from torchsummary import summary
import copy


# Define model
class CNN(nn.Module):
    """
    filters: int, out_channels = number of kernels
    units1: int, units for first linear Fully connected layer output
    units2: int, units for second linear Fully connected layer output
    input_size: tuple
    """
    def __init__(self, filters: int, units1: int, units2: int, input_size: tuple):
        super().__init__()
        self.in_channels = input_size[1]
        self.input_size = input_size

        self.convolutions = nn.Sequential(
            nn.Conv2d(self.in_channels, filters, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=0.5),
            nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=0),
            
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )

        activation_map_size = self._conv_test(self.input_size)
        print(activation_map_size)
        logger.info(f"Aggregating activationmap with size {activation_map_size}")
        self.agg = nn.AvgPool2d(activation_map_size)

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(filters, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, 32)
        )

    def _conv_test(self, input_size):
        x = torch.ones(input_size, dtype=torch.float32)
        x = self.convolutions(x)
        return x.shape[-2:]

    def forward(self, x):
        x = self.convolutions(x)
        x = self.agg(x)
        logits = self.dense(x)
        return logits


In [7]:
model = CNN(filters=128, units1=128, units2=224, input_size=(32, 3, 224, 224))
summary(model, input_size=(3, 224, 224), device="cpu")

2024-12-13 16:33:36.104 | INFO     | __main__:__init__:37 - Aggregating activationmap with size torch.Size([26, 26])


torch.Size([26, 26])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 128, 224, 224]           3,584
              ReLU-2        [-1, 128, 224, 224]               0
         MaxPool2d-3        [-1, 128, 112, 112]               0
           Dropout-4        [-1, 128, 112, 112]               0
            Conv2d-5        [-1, 128, 110, 110]         147,584
              ReLU-6        [-1, 128, 110, 110]               0
         MaxPool2d-7          [-1, 128, 55, 55]               0
            Conv2d-8          [-1, 128, 53, 53]         147,584
              ReLU-9          [-1, 128, 53, 53]               0
        MaxPool2d-10          [-1, 128, 26, 26]               0
        AvgPool2d-11            [-1, 128, 1, 1]               0
          Flatten-12                  [-1, 128]               0
           Linear-13                  [-1, 128]          16,512
             ReLU-

In [8]:
experiment_path = "mlflow_cnn2D-flowers"

In [9]:
import torch.optim as optim
from mltrainer import metrics, Trainer
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()

In [10]:
log_dir = Path("../../models/cnn").resolve()
if not log_dir.exists():
    log_dir.mkdir(parents=True)

In [11]:
from mltrainer import TrainerSettings, ReportTypes

settings = TrainerSettings(
    epochs=5,
    metrics=[accuracy],
    logdir=log_dir,
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
)
settings

epochs: 5
metrics: [Accuracy]
logdir: C:\Users\Francesca\Documents\osint\code_repo\AI\MADS-MachineLearning-FP\dev\models\cnn
train_steps: 91
valid_steps: 22
reporttypes: [<ReportTypes.TENSORBOARD: 2>, <ReportTypes.MLFLOW: 3>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.1, 'patience': 10}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 10}

In [12]:
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

using cuda


In [13]:
trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optimizer,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
    device=device,
    )

2024-12-13 16:33:36.385 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to C:\Users\Francesca\Documents\osint\code_repo\AI\MADS-MachineLearning-FP\dev\models\cnn\20241213-163336
2024-12-13 16:33:38.365 | INFO     | mltrainer.trainer:__init__:72 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.


In [14]:
trainer.loop()

  0%|                                                                                                              | 0/5 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 784.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 4.98 GiB is free. Of the allocated memory 19.72 MiB is allocated by PyTorch, and 2.28 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Use MLFLOW
Start mlflow with:

```
mlflow server     --backend-store-uri sqlite:///mlflow.db     --default-artifact-root ./mlruns     --host 127.0.0.1:5000
```

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import mlflow
import torch.optim as optim
from mltrainer import metrics, Trainer, TrainerSettings, ReportTypes
from datetime import datetime
experiment_path = "mlflow_test"
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor
from loguru import logger
from datetime import datetime

In [ ]:
#end previous run
mlflow.end_run()


In [ ]:
experiment_path = "mlflow_flowers1Dconv"
gesturesfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
batchsize = 32
preprocessor = PaddedPreprocessor()
streamers = gesturesfactory.create_datastreamer(batchsize=batchsize, preprocessor=preprocessor)

train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment(experiment_path)

In [ ]:

# Define the hyperparameter search space
settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellog",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.MLFLOW],
)


# Define the objective function for hyperparameter optimization
def objective(params):
    # Start a new MLflow run for tracking the experiment
    with mlflow.start_run():
        # Set MLflow tags to record metadata about the model and developer
        mlflow.set_tag("model", "convnet")
        mlflow.set_tag("dev", "fp")
        # Log hyperparameters to MLflow
        mlflow.log_params(params)
        print(params)
        mlflow.log_param("batchsize", f"{batchsize}")


        # Initialize the optimizer, loss function, and accuracy metric
        optimizer = optim.Adam
        loss_fn = torch.nn.CrossEntropyLoss()
        accuracy = metrics.Accuracy()

        # Instantiate the CNN model with the given hyperparameters
        model = Gesture1DCNN(**params)
        # Train the model using a custom train loop
        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optimizer,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            device=device,
        )
        trainer.loop()

        # Save the trained model with a timestamp
        tag = datetime.now().strftime("%Y%m%d-%H%M")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

        # Log the saved model as an artifact in MLflow
        mlflow.log_artifact(local_path=modelpath, artifact_path="pytorch_models")
        return {'loss' : trainer.test_loss, 'status': STATUS_OK}

In [ ]:
# search_space = {
#     'filters' : scope.int(hp.quniform('filters', 16, 128, 8)),
#     'units1' : scope.int(hp.quniform('units1', 32, 128, 8)),
#     'units2' : scope.int(hp.quniform('units2', 32, 128, 8)),
# }
search_space = {
     'filters' : scope.int(hp.quniform('filters', 16, 128, 8)),
     'units1' : scope.int(hp.quniform('units1', 32, 256, 8)),
     'units2' : scope.int(hp.quniform('units2', 32, 256, 8)),
}

In [ ]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

In [ ]:
best_result

In [ ]:
best_result

In [ ]:
best_result

In [ ]:
## Naive model